# Generowanie danych do walidacji

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

start_date = datetime(2024, 5, 13, 16, 0, 0)
end_date = datetime(2024, 5, 13, 19, 2, 37)

def generate_sample(traffic_type):
    frame_time = random_date(start_date, end_date).strftime('%Y-%m-%dT%H:%M:%S.%f%z')
    if traffic_type == "Normal":
        ip_src_host = random.randint(0, 10)
        ip_dst_host = random.randint(0, 10)
        tcp_connection_syn = random.choice([0.0, 1.0])
        tcp_connection_synack = random.choice([0.0, 1.0])
        tcp_dstport = random.uniform(0, 7000)
        tcp_len = random.uniform(0, 1500)
        tcp_seq = random.uniform(0, 500000)
        tcp_flags_index = random.uniform(0.0, 7.0)
    elif traffic_type == "DDoS_TCP":
        ip_src_host = random.randint(50000, 300000)
        ip_dst_host = random.randint(50000, 300000)
        tcp_connection_syn = random.choice([0.0, 1.0])
        tcp_connection_synack = random.choice([0.0, 1.0])
        tcp_dstport = random.uniform(1000, 20000)
        if tcp_dstport == 80.0:
            tcp_len = 120.0
            tcp_flags_index = 0.0
        else:
          tcp_len = random.choice([0.0, 120.0])
          tcp_flags_index = random.choice([0.0, 1.0])
        random.uniform(1000, 20000)
        tcp_len = random.choice([0.0, 120.0])
        tcp_seq = random.choice([0.0, 1.0])
    elif traffic_type == "Port_Scanning":
        ip_src_host = random.choice([0, 1, 2, 3, 4, 5])
        ip_dst_host = random.randint(0, 10)
        tcp_connection_syn = random.choice([0.0, 1.0])
        tcp_connection_synack = random.choice([0.0, 1.0])
        if ip_src_host == 3:
            tcp_dstport = 80.0
        else:
            tcp_dstport = random.uniform(0, 10000)
        tcp_len = 0.0
        tcp_seq = random.choice([0.0, 1.0])
        if tcp_connection_syn == 1.0:
            tcp_flags_index = 0.0
        else:
            tcp_flags_index = random.choice([1.0, 2.0])
    else:
        raise ValueError("Invalid traffic type")

    return [frame_time, ip_src_host, ip_dst_host, tcp_connection_syn, tcp_connection_synack, tcp_dstport, tcp_len, tcp_seq, traffic_type, tcp_flags_index]

def generate_random_data(num_samples):
    data = []
    for _ in range(num_samples):
        traffic_type = random.choice(["Normal", "DDoS_TCP", "Port_Scanning"])
        data.append(generate_sample(traffic_type))

    columns = ['frame-time', 'ip-src_host', 'ip-dst_host', 'tcp-connection-syn', 'tcp-connection-synack', 'tcp-dstport', 'tcp-len', 'tcp-seq', 'Attack_type', 'tcp-flags_index']
    return pd.DataFrame(data, columns=columns)

validation_data = generate_random_data(10000)
validation_data.to_csv('/content/validation-data.csv', index=False)

# Walidacja danych

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

# Wczytanie nowego zbioru danych walidacyjnych
validation_df = pd.read_csv('/content/validation-data.csv')

# Wczytanie zapisanych modelu, skalera i kodera etykiet
with open('/content/network_attack_detector_rf.pkl', 'rb') as f:
    model = pickle.load(f)
with open('/content/scaler_rf.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('/content/label_encoder_rf.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Przetwarzanie nowego zbioru danych walidacyjnych
validation_df['frame-time'] = pd.to_datetime(validation_df['frame-time']).astype(int) / 10**6
X_validation = validation_df.drop(['Attack_type'], axis=1)
y_true = label_encoder.transform(validation_df['Attack_type'])

# Normalizacja danych walidacyjnych
X_validation = scaler.transform(X_validation)

# Przewidywanie etykiet na danych walidacyjnych
y_pred_validation = model.predict(X_validation)

# Mapowanie przewidywanych etykiet na oryginalne etykiety
y_pred_labels = label_encoder.inverse_transform(y_pred_validation)

# Dodanie przewidywanych etykiet do danych walidacyjnych
validation_df['Predicted_Attack_type'] = y_pred_labels

# Wyświetlenie przykładowych wyników
print(validation_df.head())

# Ocena wyników
accuracy = accuracy_score(y_true, y_pred_validation)
report = classification_report(y_true, y_pred_validation, target_names=label_encoder.classes_)
conf_matrix = confusion_matrix(y_true, y_pred_validation)

# Wyświetlenie wyników oceny
print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)
print('Confusion Matrix:')
print(conf_matrix)

# Zapisywanie wyników oceny do pliku tekstowego
with open('/content/validation_evaluation.txt', 'w') as f:
    f.write(f'Accuracy: {accuracy:.4f}\n')
    f.write('Classification Report:\n')
    f.write(report)
    f.write('Confusion Matrix:\n')
    f.write(np.array2string(conf_matrix))

print("Ocena wyników walidacji została zapisana do pliku validation_evaluation.txt")